In [1]:
import sys
sys.path.append("../")
print(sys.executable)

d:\projects\doc-intelligence-rag\.venv\Scripts\python.exe


In [3]:
# Test chunker
from src.rag import chunk_text

text = "Machine Learning is a subset of artificial intelligence that involves training models to make predictions or decisions based on data. It is a powerful tool for solving a wide range of problems, from image recognition to natural language processing. In this article, we will explore the basics of machine learning and how it can be used to solve real-world problems."
chunks = chunk_text(text, chunk_size=12, overlap=2)
print(f"✓ Chunker works: {len(chunks)} chunks")
print(f"Split into {len(chunks)} chunks:")
for chunk in chunks:
    print(f"  Chunk {chunk.chunk_id}: {chunk.word_count} words | {chunk.text[:60]}")

Loaded .env from: d:\projects\doc-intelligence-rag\notebooks\..\src\rag\../..\.env
✓ Chunker works: 6 chunks
Split into 6 chunks:
  Chunk 0: 12 words | Machine Learning is a subset of artificial intelligence that
  Chunk 1: 12 words | training models to make predictions or decisions based on da
  Chunk 2: 12 words | It is a powerful tool for solving a wide range of problems,
  Chunk 3: 12 words | of problems, from image recognition to natural language proc
  Chunk 4: 12 words | this article, we will explore the basics of machine learning
  Chunk 5: 10 words | and how it can be used to solve real-world problems.


In [4]:
# check embeddings

from src.rag import OllamaEmbeddingClient

client = OllamaEmbeddingClient()
embedding = client.embed(text)
print(f"✓ Embeddings work: {len(embedding)} dimensions")

INFO:src.rag.embeddings:✓ Connected to Ollama at http://localhost:11434


✓ Embeddings work: 768 dimensions


In [5]:
# check vector store

from src.rag import ChromaVectorStore

store = ChromaVectorStore(persist_directory=".chromadb_test")
store.clear()
# Add chunks
store.add("chunk1", "ml", embedding, metadata={"source": "test"})

# Retrieve
results = store.retrieve(embedding, top_k=1)
for r in results:
    print(f"Chunk ID: {r.chunk_id}, Similarity: {r.similarity:.2f}, Text: {r.text}")


INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:src.rag.vector_store:✓ Initialized Chroma vector store at .chromadb_test (collection: rag)
INFO:src.rag.vector_store:Cleared vector store


Chunk ID: chunk1, Similarity: 1.00, Text: ml


In [6]:
# test groq llm
from src.rag import GroqLLMClient
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")

llm = GroqLLMClient(api_key=api_key)
answer = llm.query("Context: Hello", "Query: Who are you?")
print(f"✓ LLM works: {answer[:50]}...")

INFO:src.rag.llm:Groq LLM client initialized with model: llama-3.1-8b-instant
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


✓ LLM works: I am a helpful assistant....


In [7]:
#test full pipeline
from src.rag import RAGPipeline

pipeline = RAGPipeline()
pipeline.ingest("doc1", text)
result = pipeline.query("What will you explore?")
print(f"✓ Pipeline works!")
print(f"Answer: {result['answer']}")

INFO:src.rag.pipeline:Initializing RAG Pipeline...
INFO:src.rag.embeddings:✓ Connected to Ollama at http://localhost:11434
INFO:src.rag.pipeline:✓ Embeddings client ready
INFO:src.rag.llm:Groq LLM client initialized with model: llama-3.1-8b-instant
INFO:src.rag.pipeline:✓ LLM client ready
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:src.rag.vector_store:✓ Initialized Chroma vector store at .chromadb (collection: rag)
INFO:src.rag.pipeline:✓ Vector store ready
INFO:src.rag.pipeline:✓ RAG Pipeline initialized
INFO:src.rag.pipeline:Ingesting document: doc1
INFO:src.rag.pipeline:✓ Chunks created: 1
INFO:src.rag.pipeline:✓ Embedded 1/1 chunks
INFO:src.rag.pipeline:Querying: What will you explore?
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:src.rag.pipeline:Query complete: success


✓ Pipeline works!
Answer: Based on the provided context, it's not explicitly stated what will be explored.


In [6]:
# test pdf processor

from src.rag import RAGPipeline
import logging
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
papers_path = os.path.join(project_root, "papers")

logging.basicConfig(level=logging.INFO)

# Initialize pipeline
pipeline = RAGPipeline()

# Option A: Ingest a single PDF
# result = pipeline.ingest_pdf("papers/your_paper.pdf")
# print(f"✓ Ingested {result['chunks_embedded']} chunks")

# Option B: Ingest all PDFs from folder (RECOMMENDED)
if os.path.exists(papers_path):
    print("[1] Ingesting all PDFs from 'papers' folder...")
    results = pipeline.ingest_folder(papers_path)
    
    print(f"\n[2] Ingestion Summary:")
    total_chunks = 0
    for doc_id, result in results.items():
        print(f"  {doc_id}: {result['chunks_embedded']} chunks")
        total_chunks += result['chunks_embedded']
    print(f"  Total: {total_chunks} chunks")
    
    # Now query!
    print(f"\n[3] Querying...")
    query = "What are the main findings?"  # Change this to your question
    result = pipeline.query(query)
    
    print(f"\n📝 Question: {result['query']}")
    print(f"\n🤖 Answer:\n{result['answer']}")
    print(f"\n📚 Sources ({result['chunks_used']} chunks):")
    for source in result['sources']:
        print(f"  - [{source['similarity']:.1%}] {source['preview'][:80]}...")
else:
    print("❌ No 'papers' folder found. Create one and add PDFs!")

INFO:src.rag.pipeline:Initializing RAG Pipeline...
INFO:src.rag.embeddings:✓ Connected to Ollama at http://localhost:11434
INFO:src.rag.pipeline:✓ Embeddings client ready
INFO:src.rag.llm:Groq LLM client initialized with model: llama-3.1-8b-instant
INFO:src.rag.pipeline:✓ LLM client ready
INFO:src.rag.vector_store:✓ Initialized Chroma vector store at .chromadb (collection: rag)
INFO:src.rag.pipeline:✓ Vector store ready
INFO:src.rag.pipeline:✓ RAG Pipeline initialized
INFO:src.rag.pdf_processor:Processing folder: d:\projects\doc-intelligence-rag\papers
INFO:src.rag.pdf_processor:Found 3 PDF files
INFO:src.rag.pdf_processor:Processing PDF: CMBFSCNN.pdf


[1] Ingesting all PDFs from 'papers' folder...


INFO:src.rag.pdf_processor:✓ Extracted 26 pages, 90013 chars
INFO:src.rag.pdf_processor:Processing PDF: CVPR Version 16723_CMB_ML_A_Cosmic_Microwav.pdf
INFO:src.rag.pdf_processor:✓ Extracted 11 pages, 53068 chars
INFO:src.rag.pdf_processor:Processing PDF: Petroff20 - Cleaning CMB with ML.pdf
INFO:src.rag.pdf_processor:✓ Extracted 11 pages, 43264 chars
INFO:src.rag.pdf_processor:✓ Processed 3 PDFs successfully
INFO:src.rag.pipeline:Ingesting document: CMBFSCNN
INFO:src.rag.pipeline:✓ Chunks created: 33
INFO:src.rag.pipeline:✓ Embedded 33/33 chunks
INFO:src.rag.pipeline:Ingesting document: CVPR Version 16723_CMB_ML_A_Cosmic_Microwav
INFO:src.rag.pipeline:✓ Chunks created: 19
INFO:src.rag.pipeline:✓ Embedded 19/19 chunks
INFO:src.rag.pipeline:Ingesting document: Petroff20 - Cleaning CMB with ML
INFO:src.rag.pipeline:✓ Chunks created: 16
INFO:src.rag.pipeline:✓ Embedded 16/16 chunks
INFO:src.rag.pipeline:Querying: What are the main findings?



[2] Ingestion Summary:
  CMBFSCNN: 33 chunks
  CVPR Version 16723_CMB_ML_A_Cosmic_Microwav: 19 chunks
  Petroff20 - Cleaning CMB with ML: 16 chunks
  Total: 68 chunks

[3] Querying...


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:src.rag.pipeline:Query complete: success



📝 Question: What are the main findings?

🤖 Answer:
The provided context does not explicitly contain information about the main findings. It appears to be a collection of scientific texts discussing the Cosmic Microwave Background (CMB) and its analysis. The main topics covered include the CMB's power spectrum, contamination by foreground radiations, and the process of component separation.

📚 Sources (3 chunks):
  - [57.3%] in- frared background (CIB) is a different diffuse extragalactic source. 0 200 4...
  - [57.1%] sion [1] and LiteBird [26], are expected to probe some of the 084 deepest myster...
  - [57.0%] primordial B mode originates from the primordial gravitational waves predicted b...


In [7]:
queries = [
    "What is the Cosmic Microwave Background?",
    "How is machine learning used to analyze CMB data?",
    "What are foreground contaminations in CMB?",
    "What component separation methods are discussed?",
    "What deep learning architectures are used?",
    "What datasets are mentioned?",
]

for query in queries:
    print(f"\n{'='*60}")
    print(f"Q: {query}")
    print('='*60)
    
    result = pipeline.query(query)
    print(f"\nA: {result['answer']}")
    print(f"\nSources ({result['chunks_used']} chunks):")
    for source in result['sources'][:2]:  # Show top 2
        print(f"  [{source['similarity']:.1%}] {source['preview'][:60]}...")

INFO:src.rag.pipeline:Querying: What is the Cosmic Microwave Background?



Q: What is the Cosmic Microwave Background?


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:src.rag.pipeline:Query complete: success
INFO:src.rag.pipeline:Querying: How is machine learning used to analyze CMB data?



A: The Cosmic Microwave Background (CMB) has characteristic modes which are visible as consistently sized lumps and has a first peak at ℓ≈200, which corresponds roughly to 1 deg, about the size of the largest lumps visible in the CMB.

Sources (3 chunks):
  [60.1%] Machine learning is AI. Deep learning uses networks....
  [46.4%] in- frared background (CIB) is a different diffuse extragala...

Q: How is machine learning used to analyze CMB data?


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:src.rag.pipeline:Query complete: success
INFO:src.rag.pipeline:Querying: What are foreground contaminations in CMB?


...


A: The following component separation methods are discussed:

1. Internal Linear Combination (ILC)
2. Needlet (NILC)
3. Scale Discretized (SILC)
4. Hierarchical Morphological Component Analysis (HGMCA)
5. Convolutional Neural Network (CNN)-based methods (CMBFSCNN is specifically mentioned)

Sources (3 chunks):
  [55.4%] 497 [4] Yashar Akrami, M Ashdown, Jonathan Aumont, Carlo Bac...
  [55.0%] in several next generation CMB experiments, such as the CMB-...

Q: What deep learning architectures are used?


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:groq._base_client:Retrying request to /openai/v1/chat/completions in 23.000000 seconds
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:src.rag.pipeline:Query complete: success
INFO:src.rag.pipeline:Querying: What datasets are mentioned?



A: Based on the provided context, the following deep learning architectures are mentioned:

1. Convolutional Neural Network (CNN) - mentioned in several references, including [51], [56], and [61].
2. Multi-scale Convolutional Neural Network - mentioned in [53].
3. Spherical Convolutional Neural Network - mentioned in [56] (DeepSphere).
4. U-Net - mentioned in [61].

Note that PyTorch is also mentioned, but it is a deep learning library rather than a specific architecture.

Sources (3 chunks):
  [63.9%] Machine learning is AI. Deep learning uses networks....
  [63.3%] Advances in Neural Information Processing Systems 30, ed. I....

Q: What datasets are mentioned?


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:groq._base_client:Retrying request to /openai/v1/chat/completions in 14.000000 seconds
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:src.rag.pipeline:Query complete: success



A: Based on the provided context, the following datasets are mentioned:

1. The CMB-ML dataset, which is a unified framework for dataset generation and model evaluation.
2. The CMB-S4 science mission dataset.
3. The LiteBird dataset.
4. The COBE (Cosmic Background Explorer) dataset.
5. The Planck Mission dataset.
6. The Cosmic Infrared Background (CIB) dataset.

Sources (3 chunks):
  [62.0%] REVIEW COPY. DO NOT DISTRIBUTE. physics methods, each of the...
  [58.2%] in- frared background (CIB) is a different diffuse extragala...
